In [1]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time
import datetime as dt 
import pymongo
from pymongo import MongoClient
import numpy as np
pd.options.mode.chained_assignment = None 

In [2]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [3]:
def get_trendings(trending, start_date, number_days, minutes_spam ):
    trending_words = '(' +trending+  ') -is:retweet'
    df_list = []
    end_date = start_date + relativedelta(days=number_days)
    start_time = dt.datetime.now()
    while start_date < end_date:
        now = dt.datetime.now()
        print('Extraction Date:', start_date, 'Trending:',trending, 'Start Time',start_time,  'Time extraction:',now, end="\r")
        upper_limit = start_date+ relativedelta(minutes=minutes_spam)
        start= start_date.isoformat("T") + "Z"
        end = upper_limit.isoformat("T") + "Z"
        rsp = client.search_all_tweets(query = trending_words ,start_time = start, end_time = end, sort_order= 'relevancy',
                                     tweet_fields = fields, max_results=500)
        time.sleep(3)
        tweets_df = pd.DataFrame(rsp.data, columns = fields)
        tweets_df['date']= start
        tweets_df = pd.concat([tweets_df.drop(['public_metrics'], axis=1), pd.json_normalize(tweets_df['public_metrics'])], axis=1)
        tweets_df = pd.concat([tweets_df.drop(['geo'], axis=1), pd.json_normalize(tweets_df['geo'])], axis=1)
        df_list.append(tweets_df)
        start_date = upper_limit
    trendings_df = pd.concat(df_list)
    trendings_df = trendings_df.reset_index(drop=True)
    return(trendings_df)

In [5]:
fields = ['lang',
 'public_metrics',
 'author_id',
 'id',
 'created_at',
 'text',
         'conversation_id',
         'geo',
         'in_reply_to_user_id']
trendings_keywords = pd.read_csv('../Data/trendings.csv')
for i in range(len(trendings_keywords)):
    start_date = trendings_keywords['date'][i]
    newpath = '../Data/Trendings/'+ start_date + '/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    trending = trendings_keywords['trending'][i]
    file_name = start_date + '__' +  trending + '.gzip'
    if file_name not in os.listdir(newpath):
        number_days = 1
        minutes_spam = 10
        start_date_format = datetime.strptime(start_date, '%Y-%m-%d')
        try:
            trendings_df  = get_trendings(trending, start_date_format, number_days, minutes_spam )
        except:
            trendings_df  = get_trendings(trending, start_date_format, number_days, minutes_spam )
        trendings_df.to_parquet(newpath + file_name,
                  compression='gzip')
        trendings_keywords['Count'][i] = len(trendings_df)
    trendings_keywords.to_csv('../Data/trendings.csv', index=False)

In [ ]:
text	